In [1]:
# === SEKCJA 1: IMPORT I KONFIGURACJA ===
import pandas as pd
import numpy as np
import mlflow
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from sklearn.feature_extraction.text import TfidfVectorizer
from pycaret.classification import (
    setup, compare_models, tune_model,
    finalize_model, save_model, pull
)

# Konfiguracja MLflow
MLFLOW_EXPERIMENT_NAME = 'Investoro_District_Classification_v2'
mlflow.set_tracking_uri("http://localhost:5000") # Upewnij się, że adres jest poprawny
mlflow.set_experiment(MLFLOW_EXPERIMENT_NAME)
print(f"MLflow ustawiony. Eksperyment: '{MLFLOW_EXPERIMENT_NAME}'")

# Wczytanie danych
df_original = pd.read_csv('data.csv', sep=',')
print(f"Wczytano dane. Kształt: {df_original.shape}")

2025/06/13 15:28:08 INFO mlflow.tracking.fluent: Experiment with name 'Investoro_District_Classification_v2' does not exist. Creating a new experiment.


MLflow ustawiony. Eksperyment: 'Investoro_District_Classification_v2'
Wczytano dane. Kształt: (20092, 51)


In [2]:
# === SEKCJA 2: CZYSZCZENIE DANYCH ===
print(f"Oryginalny rozmiar danych: {df_original.shape}")
df_cleaned = df_original.copy()

# Krok 1: Usunięcie kluczowych braków danych
# Usuwamy wiersze, gdzie brakuje absolutnie podstawowych informacji
df_cleaned.dropna(subset=['Area', 'Price', 'Location', 'Description'], inplace=True)
print(f"Rozmiar po usunięciu podstawowych NaN: {df_cleaned.shape}")

# Krok 2: Usunięcie outlierów na podstawie ceny (metoda IQR)
Q1_price = df_cleaned["Price"].quantile(0.25)
Q3_price = df_cleaned["Price"].quantile(0.75)
IQR_price = Q3_price - Q1_price
df_cleaned = df_cleaned[~((df_cleaned["Price"] < (Q1_price - 1.5 * IQR_price)) | (df_cleaned["Price"] > (Q3_price + 1.5 * IQR_price)))]
print(f"Rozmiar po usunięciu outlierów z Price: {df_cleaned.shape}")

# Krok 3: Usunięcie outlierów na podstawie powierzchni (metoda IQR)
Q1_area = df_cleaned["Area"].quantile(0.25)
Q3_area = df_cleaned["Area"].quantile(0.75)
IQR_area = Q3_area - Q1_area
df_cleaned = df_cleaned[~((df_cleaned["Area"] < (Q1_area - 1.5 * IQR_area)) | (df_cleaned["Area"] > (Q3_area + 1.5 * IQR_area)))]
print(f"Rozmiar po usunięciu outlierów z Area: {df_cleaned.shape}")

# Krok 4: Konwersja daty - ważne, by zrobić to przed inżynierią cech
df_cleaned['BuiltYear'] = pd.to_datetime(df_cleaned['BuiltYear'], format='%Y', errors='coerce')

print("\nCzyszczenie danych zakończone.")

Oryginalny rozmiar danych: (20092, 51)
Rozmiar po usunięciu podstawowych NaN: (19867, 51)
Rozmiar po usunięciu outlierów z Price: (19328, 51)
Rozmiar po usunięciu outlierów z Area: (18612, 51)

Czyszczenie danych zakończone.


In [3]:
# === SEKCJA 3: INŻYNIERIA CECH (TWORZENIE FINALNEGO ZBIORU) ===
df_features = df_cleaned.copy()

# --- Krok 3a: Ekstrakcja Dzielnicy (Tworzenie naszego celu - targetu) ---
def extract_district(location_str):
    if not isinstance(location_str, str): return np.nan
    parts = [part.strip() for part in location_str.split(',')]
    if len(parts) >= 3: return f"{parts[1]}, {parts[2]}" # Np. "Warszawa, Mokotów"
    elif len(parts) == 2: return parts[1] # Np. "Kraków"
    else: return parts[0]

df_features['District'] = df_features['Location'].apply(extract_district)
df_features.dropna(subset=['District'], inplace=True)
print(f"1. Dodano kolumnę 'District'. Liczba unikalnych wartości: {df_features['District'].nunique()}")

# --- Krok 3b: Tworzenie cech TF-IDF z opisu ogłoszenia ---
vectorizer = TfidfVectorizer(max_features=100, ngram_range=(1, 2), min_df=5, max_df=0.95)

# Aby uniknąć problemów z indeksami przy łączeniu, resetujemy je
df_features.reset_index(drop=True, inplace=True)
tfidf_features = vectorizer.fit_transform(df_features['Description']) # .fillna('') już niepotrzebne
feature_names = vectorizer.get_feature_names_out()
tfidf_df = pd.DataFrame(tfidf_features.toarray(), columns=['des_tfidf_' + name for name in feature_names])

# Łączymy wszystko w jeden finalny DataFrame
df_final = pd.concat([df_features, tfidf_df], axis=1)
print(f"2. Dodano cechy TF-IDF. Finalny kształt zbioru: {df_final.shape}")

1. Dodano kolumnę 'District'. Liczba unikalnych wartości: 444
2. Dodano cechy TF-IDF. Finalny kształt zbioru: (18612, 152)


In [4]:
# === SEKCJA 4: PRZYGOTOWANIE DO MODELOWANIA ===

# --- Krok 4a: Filtrowanie rzadkich dzielnic (klas) ---
MIN_SAMPLES_PER_DISTRICT = 3
district_counts = df_final['District'].value_counts()
districts_to_remove = district_counts[district_counts < MIN_SAMPLES_PER_DISTRICT].index
df_model_ready = df_final[~df_final['District'].isin(districts_to_remove)].copy() # .copy() dla bezpieczeństwa

print(f"Usunięto {len(districts_to_remove)} rzadkich dzielnic. Pozostało {len(df_model_ready)} wierszy.")
print(f"Liczba klas (dzielnic) do predykcji: {df_model_ready['District'].nunique()}")

# --- Krok 4b: Podział na zbiór treningowy i testowy (holdout) ---
data_train = df_model_ready.sample(frac=0.9, random_state=1122)
data_test = df_model_ready.drop(data_train.index)
print(f"\nPodział danych -> Treningowe: {data_train.shape}, Testowe: {data_test.shape}")

Usunięto 132 rzadkich dzielnic. Pozostało 18430 wierszy.
Liczba klas (dzielnic) do predykcji: 312

Podział danych -> Treningowe: (16587, 152), Testowe: (1843, 152)


In [6]:
# === SEKCJA 5: SETUP PYCARET (BEZ LOGOWANIA) ===

print("\nRozpoczynam setup PyCaret (bez interakcji z MLflow)...")

# Definicja list cech
numeric_features = [c for c in data_train.columns if c.startswith('des_tfidf_')] + ['Area', 'NumberOfRooms', 'Floor', 'Floors']
categorical_features = ['BuildingType', 'BuildingCondition', 'TypeOfMarket', 'OwnerType', 'Type', 'OfferFrom', 'VoivodeshipNumber', 'CountyNumber']
date_features = ['BuiltYear']

# Automatyczne usunięcie cech, których może nie być
numeric_features = [f for f in numeric_features if f in data_train.columns]
categorical_features = [f for f in categorical_features if f in data_train.columns]

# Lista ignorowanych cech
ignore_features = [
    'SaleId', 'OriginalId', 'Location', 'Description', 'Title', 'Link', 'Price'
] + [c for c in data_train.columns if c.startswith('Date') and c != 'BuiltYear']

s = setup(
    data=data_train,
    test_data=data_test,
    target='District',
    session_id=1122,
    
    # KLUCZOWA ZMIANA: Całkowicie wyłączamy logowanie na tym etapie
    log_experiment=False, 
    
    numeric_features=numeric_features,
    categorical_features=categorical_features,
    date_features=date_features,
    ignore_features=ignore_features,
    
    # Zostawiamy dla bezpieczeństwa
    html=False,
    verbose=True # Ustawiamy na True, aby zobaczyć tabelę z podsumowaniem setup
)
print("\nSetup PyCaret zakończony pomyślnie.")


Rozpoczynam setup PyCaret (bez interakcji z MLflow)...
                    Description  \
0                    Session id   
1                        Target   
2                   Target type   
3                Target mapping   
4           Original data shape   
5        Transformed data shape   
6   Transformed train set shape   
7    Transformed test set shape   
8               Ignore features   
9              Numeric features   
10                Date features   
11         Categorical features   
12     Rows with missing values   
13                   Preprocess   
14              Imputation type   
15           Numeric imputation   
16       Categorical imputation   
17     Maximum one-hot encoding   
18              Encoding method   
19               Fold Generator   
20                  Fold Number   
21                     CPU Jobs   
22                      Use GPU   
23               Log Experiment   
24              Experiment Name   
25                          USI   

In [8]:
if mlflow.active_run():
    mlflow.end_run()

In [9]:
# === SEKCJA 6: TRENING I LOGOWANIE DO MLFLOW ===

# Upewnij się, że nie ma aktywnego runu
if mlflow.active_run():
    mlflow.end_run()

with mlflow.start_run(run_name="District_Classification_Training_Run") as run:
    print(f"Rozpoczęto run w MLflow: {run.info.run_id}")
    print("Rozpoczynam porównywanie modeli (wyniki będą logowane)...")
    
    # compare_models wykryje aktywny run i zaloguje do niego metryki i modele
    best_model = compare_models(sort='F1')
    
    print("\nPorównanie modeli zakończone.")
    
    # Wyciągnij i wyświetl tabelę
    all_models_metrics = pull()
    display(all_models_metrics)
    
    # Zaloguj tabelę jako artefakt
    all_models_metrics.to_csv("compare_models_results.csv")
    mlflow.log_artifact("compare_models_results.csv")
    
    print(f"\nNajlepszy zidentyfikowany model: {best_model}")

Rozpoczęto run w MLflow: c11fad3d1ef04f39b8f5648c43675ef3
Rozpoczynam porównywanie modeli (wyniki będą logowane)...



Porównanie modeli zakończone.


""



Najlepszy zidentyfikowany model: []


In [10]:
# === SEKCJA 7: TUNING I FINALIZACJA ===
# Uruchom w ramach tego samego bloku 'with' lub po nim,
# PyCaret powinien zalogować do ostatniego aktywnego runu.

print("\nRozpoczynam tuning najlepszego modelu...")
tuned_model = tune_model(best_model, n_iter=25, sort='F1')
print("\nTuning zakończony. Wyniki po tuningu:")
display(pull())

print("\nFinalizowanie modelu...")
final_model = finalize_model(tuned_model)
print(f"Model sfinalizowany: {final_model}")

save_model(final_model, 'district_classifier_final_model_v2')
print("\nFinalny model został zapisany do pliku.")


Rozpoczynam tuning najlepszego modelu...


ValueError: Estimator [] does not have the required fit() method.